In [ ]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import preprocessing as pp
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from sklearn.model_selection import GridSearchCV

In [ ]:
df = pd.read_csv("/Users/tristannisbet/Documents/SM/Dataframe/new/user_city_sim3.csv", index_col=0)
df

In [ ]:
df.sort_values('label_id')

In [ ]:
df2 = pd.read_csv("/Users/tristannisbet/Documents/SM/Dataframe/new/user_city_dummy.csv", index_col=0)
df2

In [ ]:
df3 = pd.read_csv("/Users/tristannisbet/Documents/SM/Dataframe/new/user_city_rank.csv", index_col=0)
df3

In [ ]:
df3.reset_index(inplace=True)

In [ ]:
df

In [ ]:
full = pd.merge(right=df3, left=df, on=['user', 'city', 'label_id'])
full

In [ ]:
#ready = full.drop(columns=["rank_x", "city"])
ready.set_index(['user', 'label_id'], inplace=True)

In [ ]:
ready

In [ ]:
full.columns

In [ ]:
trythis = df.drop(columns=['city'])

In [ ]:
trythis.rename(columns={'rank': 'top'}, inplace=True)
trythis.set_index('city_id', append=True, )

In [ ]:
def rank_from_col(x):
    if x.top==5:
       return 1
    elif x.top==4:
       return 1
    elif x.top==3:
       return 1
    elif x.top==2:
       return 1
    elif x.top==1:
       return 1
    else:
        return 0 

In [ ]:
trythis['top_true'] = trythis.apply(rank_from_col,axis=1)


In [ ]:
trythis

In [ ]:
def df_no_words(df):
    df.reset_index(inplace=True)
    idk = df.copy()
    idk.set_index(['user', 'label_id'], inplace=True)
    idk.rename(columns={'rank': 'top'}, inplace=True)
    
    return idk


In [ ]:
ready.set_index('city_id', append=True, inplace=True)

In [ ]:
ready

In [ ]:
ready.xs(22, level='user', drop_level=False)


In [ ]:
import random
index2 = random.sample(range(152), 30)

In [ ]:
index2[0]

In [ ]:
ready.xs(index2[0], level='user', drop_level=False)

In [ ]:
test_setfull = pd.DataFrame()
for i in range(len(test_users)):
    test_setfull = test_setfull.append(ready.xs(test_users[i], level='user', drop_level=False))


In [ ]:
test_setfull.index.unique(level='user')

In [ ]:
test_setfull

In [ ]:
train_setfull.index.unique(level='user')

In [ ]:
train_setfull = ready.copy()

In [ ]:
train_setfull

In [ ]:
train_setfull.reset_index(level=1, inplace=True)

In [ ]:
for i in range(len(test_users)):
    train_setfull = train_setfull.drop(train_setfull.loc[(train_setfull.index == test_users[i])].index)
    

In [ ]:
train_setfull

In [ ]:
train_setfull.set_index('label_id', append=True, inplace=True)


In [ ]:
train_set2

In [ ]:
train_set = train_set2.drop(columns=['city'])
test_set = test_set2.drop(columns=['city'])

In [ ]:
sm = SMOTE(random_state = 33)
x_train, x_test, y_train, y_test = train_test_split(ready.drop('top', axis=1),
                                                    ready.top, test_size= 0.2, random_state=12)


In [ ]:
type(y_train)

In [ ]:
sm = SMOTE(random_state = 33)

X_train_new, y_train_new = sm.fit_sample(x_train, y_train.ravel())

In [ ]:
x_train

In [ ]:
train_setfull['rank_y']

In [ ]:

y_trainfull = train_setfull['rank_y']
train_setfull.drop(columns=['rank_y'], inplace=True)


In [ ]:
y_testfull = test_setfull['rank_y']
test_setfull.drop(columns=['rank_y'], inplace=True)

In [ ]:
train_setfull = pd.read_csv('/Users/tristannisbet/Documents/SM/Dataframe/new/x_train_full.csv')
test_setfull = pd.read_csv('/Users/tristannisbet/Documents/SM/Dataframe/new/x_test_full.csv')

y_trainfull = pd.read_csv('/Users/tristannisbet/Documents/SM/Dataframe/new/y_train_full.csv')
y_testfull = pd.read_csv('/Users/tristannisbet/Documents/SM/Dataframe/new/y_test_full.csv')

In [ ]:
x_trainfull.drop(columns=['ranking_weight'], inplace=True)
x_testfull.drop(columns=['ranking_weight'], inplace=True)

In [ ]:
train_setfull

In [ ]:
x_trainfull = pd.read_csv('/Users/tristannisbet/Documents/SM/Dataframe/new/x_train_full.csv')
x_testfull = pd.read_csv('/Users/tristannisbet/Documents/SM/Dataframe/new/x_test_full.csv')

y_trainfull = pd.read_csv('/Users/tristannisbet/Documents/SM/Dataframe/new/y_train_full.csv')
y_testfull = pd.read_csv('/Users/tristannisbet/Documents/SM/Dataframe/new/y_test_full.csv')

In [ ]:
x_trainfull.set_index(['user', 'label_id'], inplace=True)
y_trainfull.set_index(['user', 'label_id'], inplace=True)
y_testfull.set_index(['user', 'label_id'], inplace=True)
x_testfull.set_index(['user', 'label_id'], inplace=True)

In [ ]:
y_trainfull

In [ ]:
train_set2 = pd.read_csv('/Users/tristannisbet/Documents/SM/Dataframe/new/x_train_sim.csv')
test_set2 = pd.read_csv('/Users/tristannisbet/Documents/SM/Dataframe/new/x_test_sim.csv')

y_train_set = pd.read_csv('/Users/tristannisbet/Documents/SM/Dataframe/new/y_train_sim.csv')
y_test_set = pd.read_csv('/Users/tristannisbet/Documents/SM/Dataframe/new/y_test_sim.csv')

In [ ]:
test_set2

In [ ]:
x_test = pd.read_csv('/Users/tristannisbet/Documents/SM/Dataframe/new/x_test.csv')
y_test = pd.read_csv('/Users/tristannisbet/Documents/SM/Dataframe/new/y_test.csv')

x_train = pd.read_csv('/Users/tristannisbet/Documents/SM/Dataframe/new/x_train.csv')
y_train = pd.read_csv('/Users/tristannisbet/Documents/SM/Dataframe/new/y_train.csv')

In [ ]:
x_test.reset_index(inplace=True)

In [ ]:
test_users = x_test.user.unique().tolist()

In [ ]:
train_users = x_train.user.unique().tolist()

In [ ]:
y_train.set_index(['user', 'label_id'], inplace=True)
x_train.set_index(['user', 'label_id'], inplace=True)
y_test.set_index(['user', 'label_id'], inplace=True)
x_test.set_index(['user', 'label_id'], inplace=True)


In [ ]:

y_trainfull = pd.Series(y_trainfull['rank_y'].values, index=y_trainfull.index)


In [ ]:
y_train_set.set_index(['user', 'city_id'], inplace=True)
train_set2.set_index(['user', 'city_id'], inplace=True)
y_test_set.set_index(['user', 'city_id'], inplace=True)
test_set2.set_index(['user', 'city_id'], inplace=True)

In [ ]:
y_train_set

In [ ]:
y_train_set = pd.Series(y_train_set['top_true'].values, index=y_train_set.index)


In [ ]:
y_train

In [ ]:
y_trains = pd.Series(y_train['top'].values, index=y_train.index)


In [ ]:
y_trains

In [ ]:
sm = SMOTE(random_state = 33)


X_train_new, y_train_new = sm.fit_sample(x_train, y_trains.ravel())

In [ ]:
nr = NearMiss()
X_train_NM, y_train_NM = nr.fit_sample(x_train, y_train)

In [ ]:
scaled_features_df

In [ ]:
train_set2

In [ ]:
train_set2.drop(columns=['top'], inplace=True)
test_set2.drop(columns=['top'], inplace=True)



### Logistic Regression

Ranks lbfgs  
Accuary is .97
But precision and recall, f1-score for 1 = 0

#### SMOTE lbfgs
P: .07  
R= .27  
f1=  .11

In [ ]:
x_test

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(solver='lbfgs', max_iter=10000).fit(x_trainfull, y_trainfull)
predictions_lr = clf.predict(x_testfull)

In [ ]:
print(classification_report(y_testfull, predictions_lr))
print(confusion_matrix(y_testfull,predictions_lr))


### KNN


#### Rank
precision: .12  
recall: .01  
f1: .01  

#### SMOTE
P:  .08  
R:  .28  
f1: .12  

In [ ]:
test_set2.reset_index(inplace=True)
train_set2

In [ ]:
train_set2.user.nunique()

In [ ]:
test_set2.user.nunique()

In [ ]:
for u in train_

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier()
neigh.fit(x_trainfull, y_trainfull)
predictions_kn = neigh.predict(x_testfull)

In [ ]:
print(classification_report(y_testfull, predictions_kn))
print(confusion_matrix(y_testfull,predictions_kn))


### SVM


#### Rank
0 for all - rbf

#### SMOTE
rbf
P= .09  
R= .64  
f1 = .15  


In [ ]:
from sklearn.svm import SVC
svclassifier = SVC(kernel='rbf')
svclassifier.fit(x_trainfull, y_trainfull)
predictions_svm = svclassifier.predict(x_testfull)


In [ ]:
print(confusion_matrix(y_testfull,predictions_svm))
print(classification_report(y_testfull,predictions_svm))

In [ ]:
test.top.value_counts()

### Decision Tree


#### rank
precision: .25  
recall: .26  
f1: .25  


#### SMOTE
P= .20  
R= 26  
F1= .22  

In [ ]:
from sklearn.tree import DecisionTreeClassifier
# clf = DecisionTreeClassifier(random_state=0)
# clf.fit(x_train, y_train)
# predictions_dt = clf.predict(x_test)



clf = DecisionTreeClassifier(random_state=0)
clf.fit(x_trainfull, y_trainfull)
predictions_dt = clf.predict(x_testfull)

In [ ]:
print(classification_report(y_testfull, predictions_dt))
print(confusion_matrix(y_testfull,predictions_dt))


### Random Forest


#### Rank
Precision: .94  
recall: .10  
f1:  .19


#### SMOTE
P= .83  
R= .13  
F1= .23  

In [ ]:
#'max_depth': 25, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100} random_state=12

from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=12, max_depth=25)
clf.fit(x_trainfull, y_trainfull)
predictions_rf = clf.predict(x_testfull)
probs_rf = clf.predict_proba(x_testfull)

In [ ]:
predictions_train = clf.predict(x_train)
probs_train = clf.predict_proba(x_train)

In [ ]:
x_train.shape

In [ ]:
print(classification_report(y_train, predictions_train))
print(confusion_matrix(y_train,predictions_train))

In [ ]:
test_set2

In [ ]:
print(classification_report(y_testfull, predictions_rf))
print(confusion_matrix(y_testfull,predictions_rf))


In [ ]:
rf_sm = RandomForestClassifier(random_state=12, max_depth=25)
rf_sm.fit(x_train_sm, y_train_sm)
predictions_rf_sm = rf_sm.predict(x_test)
probs_rf_sm = rf_sm.predict_proba(x_test)

In [ ]:
print(classification_report(y_test_set, predictions_rf_sm))
print(confusion_matrix(y_test_set,predictions_rf_sm))

### Creating accuracy metric

In [ ]:
results5 = x_test.copy()

In [ ]:
results = test_set2.copy()

In [ ]:
reultsfull = x_testfull.copy()

In [ ]:
results

In [ ]:
def split_array(a, df, predict):
    lst1 = list(a[:, 0])
    lst2 = list(a[:, 1])
    predict_lst = list(predict)
    df['prob_0'] = pd.Series(lst1, index=df.index)
    df['prob_1'] = pd.Series(lst2, index=df.index)
    df['predicted'] = pd.Series(predict_lst, index=df.index)
    
    #df = add_city(df)
    
    return df

In [ ]:
for c in results_1.columns:
    print(c)

In [ ]:
results_1 = split_array(probs_rf, reultsfull, predictions_rf)
results_1

In [ ]:
def add_city(results_df, city_df, test_set_df):
    
    # adds actual top ranked cities
    results_df = pd.merge(right=test_set_df, left=results_df, right_index=True, left_index=True)
    
    city_dict = dict(zip(city_df['label_id'], city_df['city']))
    results_df['city'] = results_df['label_id'].map(city_dict)

    
    return df


In [ ]:
test_setfull.set_index(['user', 'label_id'], inplace=True)

In [ ]:
city_df = df[['user','city', 'label_id']]

In [ ]:
results_1.index['label_id']

In [ ]:
df.drop(columns=['level_0'], inplace=True)

In [ ]:
city_df

In [ ]:
okk = add_city(results_1, city_df, test_setfull)


In [ ]:
city_df

In [ ]:
df1

In [ ]:
city_df = pd.read_csv("/Users/tristannisbet/Documents/SM/Dataframe/new/user_city_rank.csv", index_col=0)

In [ ]:
city_df

In [ ]:
results

In [ ]:
results = split_array(probs_rf, results, predictions_rf)

In [ ]:
results

In [ ]:
user3 = df3.loc[(df3.index == 3)]
user3[user3['rank'] != 0]


In [ ]:
results2.xs(3, level='user', drop_level=False).head(10)

In [ ]:
results2

In [ ]:
y_test_set

In [ ]:
# can remove
#results2.set_index(['user', 'city_id'], inplace=True)
result2 = pd.merge(right=y_test_set, left=results2, right_index=True, left_index=True)


In [ ]:
result2['sum'].hist()

#1.4 is good.

In [ ]:
result2['prob_1'].hist()

#.1 is good

In [ ]:
df

In [ ]:
df.set_index(['city_id'], append=True, inplace=True)

In [ ]:
results_city = pd.merge(right=results, left=df['city'], left_index=True, right_index=True, how='right')


In [ ]:
results_top_city = pd.merge(right=results2, left=df['city'], left_index=True, right_index=True, how='right')


In [ ]:
results_top_city

In [ ]:
top_with_with_rank.xs(90, level='user', drop_level=False)


In [ ]:
top_with_without_rank.xs(90, level='user', drop_level=False)


In [ ]:
# Create dataframe of highest 9 prob2 score for each user 
top_with_with_rank = results2.groupby('user').apply(lambda x: x.nlargest(10, 'prob_1')).droplevel(level=0)

In [ ]:
top_with_without_rank

In [ ]:
results2

In [ ]:
sim_score = pd.read_csv("/Users/tristannisbet/Documents/SM/Dataframe/new/sim_sum.csv", index_col=0)


In [ ]:
sim_score['city'] = sim_score['city'].str.replace('Kraków', 'Krakow')
sim_score['city'] = sim_score['city'].str.replace('Muğla', 'Mugla')
sim_score['city'] = sim_score['city'].str.replace('São Paulo', 'Sao Paulo')
sim_score['city'] = sim_score['city'].str.replace('Zürich', 'Zurich')
sim_score['city'] = sim_score['city'].str.replace('Düsseldorf', 'Dusseldorf')
sim_score['city'] = sim_score['city'].str.replace('Frankfurt am Main', 'Frankfurt')

In [ ]:
sim_score.reset_index(inplace=True)

In [ ]:
results5

In [ ]:
results5.reset_index(inplace=True)

In [ ]:
results5.user.unique()

In [ ]:
finished = pd.merge(right=results5, left=sim_score, on=['user', 'city'], how='right')


In [ ]:
finished

In [ ]:
accuracy = finished[['user', 'city',  'sum', 'predicted', 'top', 'prob1_group', 'prob2_group', 'food_sim', 'attraction_sim']]

In [ ]:
accuracy.rename(columns={'prob1_group': 'prob_0', 'prob2_group': 'prob_1'})

In [ ]:
accuracy.set_index(['user', 'city'], inplace=True)

In [ ]:
accuracy['sum'].describe()

In [ ]:
accuracy.nlargest(int(len(accuracy) * 0.1), 'sim_sum')


In [ ]:
accuracy[accuracy['prob2_group'] > .18].count()

In [ ]:
accuracy[accuracy['top_sum'] == 1]

In [ ]:
accuracy.rename(columns={'sum': 'sim_sum'}, inplace=True)

In [ ]:
results_top_city

In [ ]:
results_top_city.rename(columns={'sum': 'sim_sum'}, inplace=True)


In [ ]:
def add_top(x):
    if x.prob_1 > .45:
        return 1
    else:
        return 0
    
    
def add_top2(x):
    if x.sim_sum > 1.6:
        return 1
    else:
        return 0
    

In [ ]:
results_top_city['prob_top'] = results_top_city.apply(add_top,axis=1)
results_top_city['sum_top'] = results_top_city.apply(add_top2,axis=1)


In [ ]:
results_city

In [ ]:
results_top_city[(results_top_city['prob_top'] == 1) & (results_top_city['sum_top'] == 1)].count()  

In [ ]:
accuracy['sum'].describe()

* Create dataframe without the .5 added for top cities  
Merge into this one.  

select out top 5 for each user with high prob2 group - give a 1-5 rank  

select out top 5 for each user with high ranking_weight  

Is any 1-5 ranked prob2 group also a ranked 1-5 with ranking weight. count how many



### Scrap work

In [ ]:
results_top = results.groupby('user').apply(lambda x: x.nlargest(9, 'prob2_group')).droplevel(level=0)

In [ ]:
results_top.xs(109, level='user', drop_level=False)

In [ ]:
test_set_new2[test_set_new2['top'] == 1]

In [ ]:
test_set_top.city.value_counts()

In [ ]:

train_set_full = split_array(probs_train, train_set, predictions_train)
train_set_full

In [ ]:
train_set_full = pd.merge(right=train_set2, left=y_train_set, right_index=True, left_index=True)
train_set_full = pd.merge(right=train_set2, left=train_set2['city'], right_index=True, left_index=True)
train_set_top = train_set2.groupby('user').apply(lambda x: x.nlargest(9, 'prob2_group')).droplevel(level=0)


In [ ]:
user22 = split_array(one_probs, new_user)
user22

In [ ]:
sorted_user_group = user2.sort_values('prob2_group', ascending=False)
sorted_user = user2.sort_values('prob2', ascending=False)

In [ ]:
sorted_user.head(10)

In [ ]:
sorted_user_group.head(10)

In [ ]:
sorted_user[sorted_user['rank'] == 1]

In [ ]:
unine

In [ ]:
predictions_rf

In [ ]:
y_test

In [ ]:
probs_rf

In [ ]:
lst1 = list(probs_rf[:, 0])

In [ ]:
lst2 = list(probs_rf[:, 1])

In [ ]:
probs_rf[:, 1]

In [ ]:
df_y_test = pd.DataFrame(y_test)
df_y_test

In [ ]:
df_y_test['prob_new'] = pd.Series(prob, index=df_y_test.index)

In [ ]:
df_y_test['prob_1'] = pd.Series(lst1, index=df_y_test.index)
df_y_test['prob_2'] = pd.Series(lst2, index=df_y_test.index)

In [ ]:
df_y_test.sort_values('prob_2', ascending=False)

In [ ]:
df_y_test[df_y_test['prob_2'] > .47]

In [ ]:
predict_lst = list(predictions_rf)
df_y_test['predict'] = pd.Series(predict_lst, index=df_y_test.index)


In [ ]:
L = list(data)
A = list(data)
data_frame = pd.DataFrame(L) 
data_frame['A'] = pd.Series(A, index=data_frame.index)

In [ ]:
n_estimators = [100, 300, 500, 800, 1200]
max_depth = [5, 8, 15, 25, 30]
min_samples_split = [2, 5, 10, 15, 100]
min_samples_leaf = [1, 2, 5, 10] 

hyperF = dict(n_estimators = n_estimators, max_depth = max_depth,  
              min_samples_split = min_samples_split, 
             min_samples_leaf = min_samples_leaf)

gridF = GridSearchCV(clf, hyperF, cv = 2, verbose = 3, 
                      n_jobs = -1)
bestF = gridF.fit(X_train_new, y_train_new)

In [ ]:
predictions_rfcv = bestF.predict(x_test)

In [ ]:
print(classification_report(y_test, predictions_rfcv))
print(confusion_matrix(y_test,predictions_rf))


bestF.best_params_

{'max_depth': 25,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 100}
 random_state=12
 
 
 P= .66  
 R= .13  
 F1 = .22  
 
 
 
 4074  4  
 126   19  

In [ ]:
clf.get_params()

### Adding continents to city

In [ ]:
df3

In [ ]:
from pycountry_convert import country_alpha2_to_continent_code, country_name_to_country_alpha2


In [ ]:
continents = {
    'NA': 'North America',
    'SA': 'South America', 
    'AS': 'Asia',
    'OC': 'Australia',
    'AF': 'Africa',
    'EU': 'Europe'
}

cities['continent'] = [continents[country_alpha2_to_continent_code(country_name_to_country_alpha2(country))] for country in cities['country']]

In [ ]:
cities

In [ ]:

continent_dict = dict(zip(cities['city'], cities['continent']))
df3['continent'] = df3['city'].map(continent_dict)

In [ ]:
df3['continent'] = df3['city'].map(continent_dict)

In [ ]:
df3

In [ ]:
def add_continents(x):
    x.city in continent_dict
    
    return continent_dict[x.city]

In [ ]:
df3['continent'] = 
trythis['top_true'] = trythis.apply(rank_from_col,axis=1)


In [ ]:
df3

In [ ]:
continent_dict['London']

In [ ]:
import sqlite3


In [ ]:
def get_df(table_name):
    try:
        conn = sqlite3.connect('/Users/tristannisbet/Documents/travel_app/places.db')

    except Exception as e:
        print('Error durring connection: ', str(e))
    
    sql = """select * from {}""".format(table_name)
    df = pd.read_sql_query(sql, conn)

    return df

In [ ]:
cities = get_df('cities')

cities

In [ ]:

cities.continent.value_counts()

In [ ]:
df_city = df3['city']
df_city

In [ ]:
ok = answer(phone_number)
ok

In [ ]:
type(ok)

In [ ]:
numeric_numbe = []
type(numeric_numbe)


In [ ]:
number_dict['zero']

In [ ]:
phone_number.split()

In [ ]:
phone_number = 'one two three 4 5 six 7'
output = '1234567'

number_dict = {'zero': '0', 'one': '1', 'two': '2', 'three': '3', 'four': '4', 'five': '5',
              'six': '6', 'seven': '7', 'eight': '8', 'nine': '9'}
def answer(phone_number):
    number_list = phone_number.split()
    numeric_number = ''
    for p in number_list:
        if p in number_dict.keys():
            numeric_number += (number_dict[p])
            
        else:
            numeric_number += p
            
    
        
        
    return numeric_number
